# P&L vs Duration Correlation Analysis

This notebook analyzes the relationship between position holding duration and trade profitability to identify if the strategy benefits from quick exits or longer holds.

## Key Questions

- Do longer-held positions tend to be more or less profitable?
- Is there an optimal holding time for maximizing returns?
- Does the correlation differ across different run configurations?
- Should the strategy implement time-based exits or stop-losses?

## Features

- Scatter plots showing P&L vs duration for each trade
- Correlation coefficients for each run
- Statistical analysis of duration vs profitability
- Comparative insights across multiple configurations

## Usage

1. Configure stock symbol
2. Scan and filter runs
3. Load selected runs
4. Analyze P&L vs duration relationships

Use this notebook to understand if your strategy suffers from holding losers too long or cutting winners too early.

In [ ]:
# Import required libraries
import sys
from pathlib import Path

# Add analysis folder to path for imports
project_root = Path('/home/pjpr/projects/wee_hedgy_thing/quiescence')
analysis_path = project_root / 'analysis'

if str(analysis_path) not in sys.path:
    sys.path.insert(0, str(analysis_path))

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go

# Import custom utility functions from analysis folder
from utilities import (
    scan_backtest_runs, 
    load_run_data, 
    create_runs_summary_dataframe, 
    convert_utc_to_ny,
    build_cumulative_pnl_from_positions
)

## 1. Configuration

Set the stock symbol and paths for analysis.

In [ ]:
# === CONFIGURATION ===
stock_symbol = "MSFT"

# Project paths
STORAGE_ROOT = Path("/data/quiescence/")
BACKTEST_ROOT = STORAGE_ROOT / "backtest"

print(f"Analyzing stock: {stock_symbol}")
print(f"Backtest runs root: {BACKTEST_ROOT}")

## 2. Scan and Filter Runs

Scan all available runs and create a summary DataFrame for filtering.

In [ ]:
# Scan all runs for the configured stock (fast - metadata only)
runs_metadata = scan_backtest_runs(BACKTEST_ROOT, stock_symbol)

# Create summary DataFrame for easy filtering and comparison
df_summary = create_runs_summary_dataframe(runs_metadata)

print(f"\nFound {len(df_summary)} runs for {stock_symbol}\n")
print(df_summary.to_string(index=False))

## 3. Select Runs to Load

Choose which runs to analyze.

In [ ]:
# Option 1: Load ALL runs
# runs_data = [load_run_data(run) for run in runs_metadata]

# Option 2: Load specific runs by Run number
#selected_run_numbers = [11, 19, 23, 34, 42, 43, 47]
#runs_data = [load_run_data(runs_metadata[run_num - 1]) for run_num in selected_run_numbers if 0 < run_num <= len(runs_metadata)]

# Option 3: Filter using DataFrame conditions
# Example: Load runs with specific parameter values
#filtered_df = df_summary[
#    (df_summary['Entry P Top'].astype(float) >= 0.95) & 
#    (df_summary['Frequency'] == '1-MINUTE')
#]

filtered_df = df_summary[(df_summary['Max Pos Bars'] == 9999)]

selected_indices = filtered_df['Run'].values - 1
runs_data = [load_run_data(runs_metadata[i]) for i in selected_indices]

print(f"\nLoaded {len(runs_data)} runs:\n")
print(filtered_df.to_string(index=False))

## 4. Helper Function: Extract Position Data

This function formats position data from the positions report for analysis.

In [ ]:
def extract_positions_from_report(positions_report, debug=False):
    """
    Extract and format position data from positions report for analysis.
    
    Args:
        positions_report: DataFrame with positions data including duration, realized_pnl, timestamps
        debug: If True, print debugging information
        
    Returns:
        DataFrame with columns: entry_ts, exit_ts, duration_minutes, trade_pnl
    """
    if positions_report.empty or 'realized_pnl' not in positions_report.columns:
        return pd.DataFrame()
    
    if debug:
        print(f"Columns available: {positions_report.columns.tolist()}")
        print(f"Row count: {len(positions_report)}")
    
    # Create formatted dataframe
    df_positions = pd.DataFrame()
    
    # Calculate duration in minutes
    # First check for duration_ns (duration in nanoseconds)
    if 'duration_ns' in positions_report.columns:
        duration_col = positions_report['duration_ns']
        
        if debug:
            print(f"Using duration_ns column")
            print(f"Duration_ns dtype: {duration_col.dtype}")
            print(f"Duration_ns sample values: {duration_col.head()}")
        
        # Convert nanoseconds to minutes
        df_positions['duration_minutes'] = pd.to_numeric(duration_col, errors='coerce') / 1e9 / 60
        
    # Check for duration column (legacy support)
    elif 'duration' in positions_report.columns:
        duration_col = positions_report['duration']
        
        if debug:
            print(f"Using duration column")
            print(f"Duration column dtype: {duration_col.dtype}")
        
        if pd.api.types.is_timedelta64_dtype(duration_col):
            df_positions['duration_minutes'] = duration_col.dt.total_seconds() / 60
        elif pd.api.types.is_numeric_dtype(duration_col):
            # Assume nanoseconds
            df_positions['duration_minutes'] = duration_col / 1e9 / 60
        else:
            # Try to parse as timedelta string or numeric
            try:
                df_positions['duration_minutes'] = pd.to_timedelta(duration_col).dt.total_seconds() / 60
            except:
                df_positions['duration_minutes'] = pd.to_numeric(duration_col, errors='coerce') / 1e9 / 60
    
    # Fallback: calculate from timestamps
    elif 'ts_opened' in positions_report.columns and 'ts_closed' in positions_report.columns:
        if debug:
            print("Calculating duration from ts_opened and ts_closed")
        
        ts_opened = pd.to_numeric(positions_report['ts_opened'], errors='coerce')
        ts_closed = pd.to_numeric(positions_report['ts_closed'], errors='coerce')
        duration_ns = ts_closed - ts_opened
        df_positions['duration_minutes'] = duration_ns / 1e9 / 60
    
    else:
        if debug:
            print("Warning: No duration column or timestamps available")
        df_positions['duration_minutes'] = 0
    
    # Extract P&L - handle both string format ("65.62 USD") and numeric
    pnl_col = positions_report['realized_pnl']
    if pnl_col.dtype == 'object':
        df_positions['trade_pnl'] = pnl_col.str.replace(' USD', '').astype(float)
    else:
        df_positions['trade_pnl'] = pnl_col
    
    if debug:
        print(f"Rows before dropna: {len(df_positions)}")
        print(f"Duration NaN count: {df_positions['duration_minutes'].isna().sum()}")
        print(f"P&L NaN count: {df_positions['trade_pnl'].isna().sum()}")
        if len(df_positions) > 0:
            print(f"Duration sample values: {df_positions['duration_minutes'].head()}")
    
    # Drop rows with missing data
    df_positions = df_positions.dropna(subset=['duration_minutes', 'trade_pnl'])
    
    if debug:
        print(f"Rows after dropna: {len(df_positions)}")
    
    return df_positions

## 5. P&L vs Duration Correlation Visualization

Create scatter plots showing the relationship between position duration and profitability.

In [ ]:
# Use matplotlib colors for distinguishing runs
colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']

# Configuration for visualization
MAX_DURATION_MINUTES = 50  # Maximum x-axis limit (5 hours)

correlation_stats = []
plot_data = []  # Store data for plotting

# First pass: collect all data and statistics
for i, run in enumerate(runs_data):
    positions_report = run['positions_report']
    
    # Extract bar frequency for labeling
    bar_type = run.get('bar_type', '')
    if bar_type:
        parts = bar_type.split('-')
        frequency = f"{parts[1]}-{parts[2]}" if len(parts) >= 3 else 'Unknown'
    else:
        frequency = 'Unknown'
    
    # Check if we have positions data
    if positions_report.empty or 'realized_pnl' not in positions_report.columns:
        print(f"Warning: Run {i+1} missing positions data or 'realized_pnl' column")
        continue
    
    # Extract positions from report (enable debug for first run)
    df_positions = extract_positions_from_report(positions_report, debug=(i==0))
    
    if len(df_positions) == 0:
        print(f"Warning: Run {i+1} has no positions extracted from report")
        continue
    
    # Calculate correlation
    correlation = df_positions['duration_minutes'].corr(df_positions['trade_pnl'])
    
    # Get trading hours for label
    trading_hours = run.get("trading_hours", {})
    if isinstance(trading_hours, dict):
        start_time = trading_hours.get("start", "N/A")
        if hasattr(start_time, 'strftime'):
            start_time = start_time.strftime('%H:%M')
    else:
        start_time = run.get("trading_start_time", "N/A")
        if hasattr(start_time, 'strftime'):
            start_time = start_time.strftime('%H:%M')
    
    max_pos_bars = run.get('max_position_bars', 'N/A')
    
    # Store plot data
    plot_data.append({
        'run_num': i + 1,
        'df': df_positions,
        'frequency': frequency,
        'max_pos_bars': max_pos_bars,
        'correlation': correlation,
        'color': colors[i % len(colors)]
    })
    
    # Store correlation statistics
    correlation_stats.append({
        'Run': i + 1,
        'Frequency': frequency,
        'Max Pos Bars': max_pos_bars,
        'Start Time': start_time,
        'Trades': len(df_positions),
        'Correlation': correlation,
        'Interpretation': 'Negative' if correlation < 0 else 'Positive',
        'Strength': 'Strong' if abs(correlation) > 0.5 else 'Moderate' if abs(correlation) > 0.3 else 'Weak'
    })

# Second pass: create subplots
n_runs = len(plot_data)
if n_runs > 0:
    # Calculate grid dimensions (prefer wider layouts)
    n_cols = min(3, n_runs)  # Maximum 3 columns
    n_rows = (n_runs + n_cols - 1) // n_cols  # Ceiling division
    
    # Create figure with subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 4*n_rows))
    
    # Handle case where axes might not be a 2D array
    if n_runs == 1:
        axes = [[axes]]
    elif n_rows == 1:
        axes = [axes]
    elif n_cols == 1:
        axes = [[ax] for ax in axes]
    
    # Plot each run
    for idx, data in enumerate(plot_data):
        row = idx // n_cols
        col = idx % n_cols
        ax = axes[row][col]
        
        df = data['df']
        
        # Create scatter plot
        ax.scatter(
            df['duration_minutes'],
            df['trade_pnl'],
            c=data['color'],
            s=20,
            alpha=0.5,
            edgecolors='darkgray',
            linewidth=0.5
        )
        
        # Add horizontal line at y=0
        ax.axhline(y=0, linestyle='--', color='gray', linewidth=1, alpha=0.7)
        
        # Set title and labels
        title = f"Run {data['run_num']}: {data['frequency']}\n(Max:{data['max_pos_bars']}) - Corr: {data['correlation']:.3f}"
        ax.set_title(title, fontsize=10, fontweight='bold')
        ax.set_xlabel('Duration (minutes)', fontsize=9)
        ax.set_ylabel('P&L ($)', fontsize=9)
        
        # Set x-axis limit
        ax.set_xlim(0, MAX_DURATION_MINUTES)
        
        # Add grid
        ax.grid(True, alpha=0.3)
    
    # Hide any unused subplots
    for idx in range(n_runs, n_rows * n_cols):
        row = idx // n_cols
        col = idx % n_cols
        axes[row][col].set_visible(False)
    
    # Add overall title
    fig.suptitle(f'{stock_symbol} - Trade P&L vs Duration Correlation', 
                 fontsize=14, fontweight='bold', y=0.995)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available to plot")


## 6. Correlation Statistics Table

Display correlation coefficients and interpretation for each run.

In [ ]:
# Display correlation statistics table
if correlation_stats:
    df_correlation = pd.DataFrame(correlation_stats)
    print(f"\n{'='*100}")
    print(f"P&L vs DURATION CORRELATION ANALYSIS - {stock_symbol}")
    print(f"{'='*100}\n")
    print(df_correlation.to_string(index=False))
    print(f"\n{'='*100}")
    
    # Summary insights
    negative_corr_count = sum(1 for stat in correlation_stats if stat['Correlation'] < 0)
    total_runs = len(correlation_stats)
    
    print(f"\n📊 SUMMARY INSIGHTS:\n")
    print(f"  - {negative_corr_count}/{total_runs} runs show negative correlation (longer = less profitable)")
    print(f"  - {total_runs - negative_corr_count}/{total_runs} runs show positive correlation (longer = more profitable)")
    
    avg_correlation = sum(stat['Correlation'] for stat in correlation_stats) / total_runs
    print(f"  - Average correlation across all runs: {avg_correlation:.3f}")
    
    if avg_correlation < -0.3:
        print(f"\n  ⚠️  STRONG NEGATIVE PATTERN: Longer trades consistently lose money!")
        print(f"      💡 RECOMMENDATION: Implement tighter stop-losses or time-based exits.")
        print(f"      💡 Consider reducing max_position_bars parameter.")
    elif avg_correlation < 0:
        print(f"\n  ⚠️  WEAK NEGATIVE PATTERN: Slight tendency for longer trades to underperform.")
        print(f"      💡 RECOMMENDATION: Review and potentially tighten exit conditions.")
    elif avg_correlation > 0.3:
        print(f"\n  ✅ POSITIVE PATTERN: Longer trades tend to be more profitable.")
        print(f"      💡 RECOMMENDATION: Strategy benefits from patience and letting winners run.")
        print(f"      💡 Consider allowing more time before forced exits.")
    else:
        print(f"\n  ℹ️  WEAK/NO CORRELATION: Position duration has minimal impact on profitability.")
        print(f"      💡 RECOMMENDATION: Focus on entry/exit conditions rather than holding time.")
    
    print(f"\n{'='*100}\n")
else:
    print("\n⚠️  No correlation data available - check that runs have valid orders data")

## 7. Duration Distribution Analysis

Analyze the distribution of position durations to understand typical holding times.

In [ ]:
# Analyze duration distributions for each run
duration_dist_stats = []

for i, run in enumerate(runs_data):
    positions_report = run['positions_report']
    
    # Extract bar frequency
    bar_type = run.get('bar_type', '')
    if bar_type:
        parts = bar_type.split('-')
        frequency = f"{parts[1]}-{parts[2]}" if len(parts) >= 3 else 'Unknown'
    else:
        frequency = 'Unknown'
    
    # Check if we have positions data
    if positions_report.empty or 'realized_pnl' not in positions_report.columns:
        continue
    
    # Extract positions from report
    df_positions = extract_positions_from_report(positions_report)
    
    if len(df_positions) == 0:
        continue
    
    # Calculate duration statistics
    durations = df_positions['duration_minutes']
    
    duration_dist_stats.append({
        'Run': i + 1,
        'Frequency': frequency,
        'Max Pos Bars': run.get('max_position_bars', 'N/A'),
        'Trades': len(df_positions),
        'Avg Duration (min)': f"{durations.mean():.1f}",
        'Median Duration (min)': f"{durations.median():.1f}",
        'Std Duration (min)': f"{durations.std():.1f}",
        'Min Duration (min)': f"{durations.min():.1f}",
        'Max Duration (min)': f"{durations.max():.1f}",
        '25th Percentile (min)': f"{durations.quantile(0.25):.1f}",
        '75th Percentile (min)': f"{durations.quantile(0.75):.1f}",
    })

# Display duration distribution statistics
if duration_dist_stats:
    df_duration_dist = pd.DataFrame(duration_dist_stats)
    print("\n" + "="*120)
    print("POSITION DURATION DISTRIBUTION ANALYSIS")
    print("="*120 + "\n")
    print(df_duration_dist.to_string(index=False))
    print("\n" + "="*120)

## 8. Profitability by Duration Buckets

Analyze profitability across different duration buckets to find optimal holding times.

In [ ]:
# Analyze profitability by duration buckets
for i, run in enumerate(runs_data):
    positions_report = run['positions_report']
    
    # Extract bar frequency
    bar_type = run.get('bar_type', '')
    if bar_type:
        parts = bar_type.split('-')
        frequency = f"{parts[1]}-{parts[2]}" if len(parts) >= 3 else 'Unknown'
    else:
        frequency = 'Unknown'
    
    # Check if we have positions data
    if positions_report.empty or 'realized_pnl' not in positions_report.columns:
        continue
    
    # Extract positions from report
    df_positions = extract_positions_from_report(positions_report)
    
    if len(df_positions) == 0:
        continue
    
    # Create duration buckets
    df_positions['duration_bucket'] = pd.cut(
        df_positions['duration_minutes'],
        bins=[0, 15, 30, 60, 120, 240, float('inf')],
        labels=['0-15min', '15-30min', '30-60min', '1-2hr', '2-4hr', '4hr+']
    )
    
    # Calculate statistics for each bucket
    bucket_stats = df_positions.groupby('duration_bucket').agg({
        'trade_pnl': ['count', 'mean', 'sum', lambda x: (x > 0).sum()],
        'duration_minutes': 'mean'
    }).round(2)
    
    bucket_stats.columns = ['Trades', 'Avg P&L', 'Total P&L', 'Winners', 'Avg Duration']
    bucket_stats['Win Rate (%)'] = (bucket_stats['Winners'] / bucket_stats['Trades'] * 100).round(1)
    
    print(f"\n{'='*100}")
    print(f"Run {i+1}: {run.get('ticker', stock_symbol)} | {frequency} | Max Pos Bars: {run.get('max_position_bars', 'N/A')}")
    print(f"{'='*100}\n")
    print(bucket_stats.to_string())
    print(f"\n{'='*100}")
    
    # Identify best bucket
    best_bucket = bucket_stats['Avg P&L'].idxmax()
    print(f"\n💰 Most profitable duration bucket: {best_bucket} (Avg P&L: ${bucket_stats.loc[best_bucket, 'Avg P&L']:.2f})")